# Building the dataset of research papers

The [Entrez](http://biopython.org/DIST/docs/api/Bio.Entrez-module.html) module, a part of the [Biopython](http://biopython.org/) library, will be used to interface with [PubMed](http://www.ncbi.nlm.nih.gov/pubmed).<br>
You can download Biopython from [here](http://biopython.org/wiki/Download).

In this notebook we will be covering several of the steps taken in the [Biopython Tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html), specifically in [Chapter 9  Accessing NCBI’s Entrez databases](http://biopython.org/DIST/docs/tutorial/Tutorial.html#htoc109).

In [13]:
from Bio import Entrez

# NCBI requires you to set your email address to make use of NCBI's E-utilities
Entrez.email = "Your.Name.Here@example.org"

The datasets will be saved as serialized Python objects, compressed with bzip2.
Saving/loading them will therefore require the [pickle](http://docs.python.org/3/library/pickle.html) and [bz2](http://docs.python.org/3/library/bz2.html) modules.

In [14]:
import pickle, bz2, os

## EInfo: Obtaining information about the Entrez databases

In [15]:
# accessing extended information about the PubMed database
pubmed = Entrez.read( Entrez.einfo(db="pubmed"), validate=False )[u'DbInfo']

# list of possible search fields for use with ESearch:
search_fields = { f['Name']:f['Description'] for f in pubmed["FieldList"] }

In search_fields, we find 'TIAB' ('Free text associated with Abstract/Title') as a possible search field to use in searches.

In [16]:
search_fields

{'AFFL': "Author's institutional affiliation and address",
 'ALL': 'All terms from all searchable fields',
 'AUCL': 'Author Cluster ID',
 'AUID': 'Author Identifier',
 'AUTH': 'Author(s) of publication',
 'BOOK': 'ID of the book that contains the document',
 'CDAT': 'Date of completion',
 'CNTY': 'Country of publication',
 'COLN': 'Corporate Author of publication',
 'CRDT': 'Date publication first accessible through Entrez',
 'DSO': 'Additional text from the summary',
 'ECNO': 'EC number for enzyme or CAS registry number',
 'ED': "Section's Editor",
 'EDAT': 'Date publication first accessible through Entrez',
 'EID': 'Extended PMID',
 'EPDT': 'Date of Electronic publication',
 'FAUT': 'First Author of publication',
 'FILT': 'Limits the records',
 'FINV': 'Full name of investigator',
 'FULL': 'Full Author Name(s) of publication',
 'GRNT': 'NIH Grant Numbers',
 'INVR': 'Investigator',
 'ISBN': 'ISBN',
 'ISS': 'Issue number of publication',
 'JOUR': 'Journal abbreviation of publication',


## ESearch: Searching the Entrez databases

To have a look at the kind of data we get when searching the database, we'll perform a search for papers authored by Haasdijk:

In [17]:
example_authors = ['Haasdijk E']
example_search = Entrez.read( Entrez.esearch( db="pubmed", term=' AND '.join([a+'[AUTH]' for a in example_authors]) ) )
example_search

{'RetStart': '0', 'TranslationSet': [], 'TranslationStack': [{'Field': 'Author', 'Explode': 'N', 'Count': '31', 'Term': 'Haasdijk E[Author]'}, 'GROUP'], 'QueryTranslation': 'Haasdijk E[Author]', 'RetMax': '20', 'IdList': ['26933487', '24977986', '24901702', '24852945', '24708899', '24252306', '23580075', '23144668', '22174697', '22154920', '21870131', '21760539', '20662596', '20602234', '20386726', '18579581', '18305242', '17913916', '17804640', '17686042'], 'Count': '31'}

Note how the result being produced is not in Python's native string format:

In [18]:
type( example_search['IdList'][0] )

Bio.Entrez.Parser.StringElement

The part of the query's result we are most interested in is accessible through

In [19]:
example_ids = [ int(id) for id in example_search['IdList'] ]
print(example_ids)

[26933487, 24977986, 24901702, 24852945, 24708899, 24252306, 23580075, 23144668, 22174697, 22154920, 21870131, 21760539, 20662596, 20602234, 20386726, 18579581, 18305242, 17913916, 17804640, 17686042]


### PubMed IDs dataset

We will now assemble a dataset comprised of research articles containing the keyword "evolution", in either their titles or abstracts.

In [20]:
search_term = 'malaria'

In [21]:
Ids_file = 'data/' + search_term + '__Ids.pkl.bz2'

In [22]:
if os.path.exists( Ids_file ):
    Ids = pickle.load( bz2.BZ2File( Ids_file, 'rb' ) )
else:
    # determine the number of hits for the search term
    search = Entrez.read( Entrez.esearch( db="pubmed", term=search_term+'[TIAB]', retmax=0 ) )
    total = int( search['Count'] )
    
    # `Ids` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Ids_str = []
    retrieve_per_query = 10000
    
    for start in range( 0, total, retrieve_per_query ):
        print('Fetching IDs of results [%d,%d]' % ( start, start+retrieve_per_query ) )
        s = Entrez.read( Entrez.esearch( db="pubmed", term=search_term+'[TIAB]', retstart=start, retmax=retrieve_per_query ) )
        Ids_str.extend( s[ u'IdList' ] )
    
    # convert Ids to integers (and ensure that the conversion is reversible)
    Ids = [ int(id) for id in Ids_str ]
    
    for (id_str, id_int) in zip(Ids_str, Ids):
        if str(id_int) != id_str:
            raise Exception('Conversion of PubMed ID %s from string to integer it not reversible.' % id_str )
    
    # Save list of Ids
    pickle.dump( Ids, bz2.BZ2File( Ids_file, 'wb' ) )
    
total = len( Ids )
print('%d documents contain the search term "%s".' % ( total, search_term ) )

67028 documents contain the search term "malaria".


Taking a look at what we just retrieved, here are the last 5 elements of the `Ids` list:

In [23]:
Ids[:5]

[27749938, 27749907, 27748596, 27748303, 27748294]

## ESummary: Retrieving summaries from primary IDs

To have a look at the kind of metadata we get from a call to `Entrez.esummary()`, we now fetch the summary of one of Haasdijk's papers (using one of the PubMed IDs we obtained in the previous section:

In [12]:
example_paper = Entrez.read( Entrez.esummary(db="pubmed", id='27749938') )[0]

def print_dict( p ):
    for k,v in p.items():
        print(k)
        print('\t', v)

print_dict(example_paper)

RecordStatus
	 Unknown status
History
	 {'received': '2016/06/09 00:00', 'pubmed': ['2016/10/18 06:00'], 'accepted': '2016/09/29 00:00', 'medline': ['2016/10/18 06:00']}
DOI
	 10.1371/journal.pone.0164685
ESSN
	 1932-6203
AuthorList
	 ['Adde A', 'Roux E', 'Mangeas M', 'Dessay N', 'Nacher M', 'Dusfour I', 'Girod R', 'Briolant S']
PmcRefCount
	 0
NlmUniqueID
	 101285081
LastAuthor
	 Briolant S
References
	 []
LangList
	 ['English']
EPubDate
	 2016 Oct 17
Source
	 PLoS One
PubStatus
	 
PubTypeList
	 ['Journal Article']
HasAbstract
	 1
FullJournalName
	 PloS one
Item
	 []
PubDate
	 2016 Oct 17
Pages
	 e0164685
Issue
	 10
SO
	 2016 Oct 17;11(10):e0164685
ELocationID
	 doi: 10.1371/journal.pone.0164685
Title
	 Dynamical Mapping of Anopheles darlingi Densities in a Residual Malaria Transmission Area of French Guiana by Using Remote Sensing and Meteorological Data.
ISSN
	 
ArticleIds
	 {'eid': '27749938', 'pii': 'PONE-D-16-23217', 'pubmed': ['27749938'], 'rid': '27749938', 'medline': [], 'doi'

For now, we'll keep just some basic information for each paper: title, list of authors, publication year, and [DOI](https://en.wikipedia.org/wiki/Digital_object_identifier).

In case you are not familiar with the DOI system, know that the paper above can be accessed through the link [http://dx.doi.org/10.1007/s12065-012-0071-x](http://dx.doi.org/10.1007/s12065-012-0071-x) (which is `http://dx.doi.org/` followed by the paper's DOI).

In [13]:
( example_paper['Title'], example_paper['AuthorList'], int(example_paper['PubDate'][:4]), example_paper['DOI'] )

('Dynamical Mapping of Anopheles darlingi Densities in a Residual Malaria Transmission Area of French Guiana by Using Remote Sensing and Meteorological Data.',
 ['Adde A', 'Roux E', 'Mangeas M', 'Dessay N', 'Nacher M', 'Dusfour I', 'Girod R', 'Briolant S'],
 2016,
 '10.1371/journal.pone.0164685')

### Summaries dataset

We are now ready to assemble a dataset containing the summaries of all the paper `Ids` we previously fetched.

To reduce the memory footprint, and to ensure the saved datasets won't depend on Biopython being installed to be properly loaded, values returned by `Entrez.read()` will be converted to their corresponding native Python types. We start by defining a function for helping with the conversion of strings:

In [14]:
Summaries_file = 'data/' + search_term + '__Summaries.pkl.bz2'

In [15]:
if os.path.exists( Summaries_file ):
    Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
else:
    # `Summaries` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Summaries = []
    retrieve_per_query = 500
    
    print('Fetching Summaries of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        s = Entrez.read( Entrez.esummary( db="pubmed", id=query_ids ) )
        
        # out of the retrieved data, we will keep only a tuple (title, authors, year, DOI), associated with the paper's id.
        # (all values converted to native Python formats)
        f = [
            ( int( p['Id'] ), (
                str( p['Title'] ),
                [ str(a) for a in p['AuthorList'] ],
                int( p['PubDate'][:4] ),                # keeps just the publication year
                str( p.get('DOI', '') )            # papers for which no DOI is available get an empty string in their place
                ) )
            for p in s
            ]
        Summaries.extend( f )
    
    # Save Summaries, as a dictionary indexed by Ids
    Summaries = dict( Summaries )
    
    pickle.dump( Summaries, bz2.BZ2File( Summaries_file, 'wb' ) )

Fetching Summaries of results: 

0...................
10000...................
20000...................
30000...................
40000...................
50000...................
60000..............

Let us take a look at the first 3 retrieved summaries:

In [16]:
{ id : Summaries[id] for id in Ids[:3] }

{27748596: ('Identification of a potential anti-malarial drug candidate from a series of 2-aminopyrazines by optimization of aqueous solubility and potency across the parasite life-cycle.',
  ['Le Manach C',
   'Nchinda AT',
   'Paquet T',
   'Gonzalez Cabrera D',
   'Younis Adam Y',
   'Han Z',
   'Bashyam S',
   'Zabiulla M',
   'Taylor D',
   'Lawrence N',
   'White KL',
   'Charman SA',
   'Waterson D',
   'Witty MJ',
   'Wittlin S',
   'Botha ME',
   'Nondaba SH',
   'Reader J',
   'Birkholtz LM',
   'Jimenez-Diaz MB',
   'Martínez-Martínez MS',
   'Ferrer-Bazaga S',
   'Angulo-Barturen I',
   'Meister S',
   'Antonova-Koch Y',
   'Winzeler EA',
   'Street LJ',
   'Chibale K'],
  2016,
  '10.1021/acs.jmedchem.6b01265'),
 27749907: ('Safety and Immunogenicity of Pfs25-EPA/Alhydrogel®, a Transmission Blocking Vaccine against Plasmodium falciparum: An Open Label Study in Malaria Naïve Adults.',
  ['Talaat KR',
   'Ellis RD',
   'Hurd J',
   'Hentrich A',
   'Gabriel E',
   'Hynes NA'

## EFetch: Downloading full records from Entrez

`Entrez.efetch()` is the function that will allow us to obtain paper abstracts. Let us start by taking a look at the kind of data it returns when we query PubMed's database.

In [17]:
q = Entrez.read( Entrez.efetch(db="pubmed", id='27749938', retmode="xml") )

`q` is a list, with each member corresponding to a queried id. Because here we only queried for one id, its results are then in `q[0]`.

In [18]:
type(q), len(q)

(Bio.Entrez.Parser.ListElement, 1)

At `q[0]` we find a dictionary containing two keys, the contents of which we print below.

In [19]:
type(q[0]), q[0].keys()

(Bio.Entrez.Parser.DictionaryElement,
 dict_keys(['MedlineCitation', 'PubmedData']))

In [20]:
print_dict( q[0][ 'PubmedData' ] )

History
	 [DictElement({'Year': '2016', 'Month': '6', 'Day': '9'}, attributes={'PubStatus': 'received'}), DictElement({'Year': '2016', 'Month': '9', 'Day': '29'}, attributes={'PubStatus': 'accepted'}), DictElement({'Hour': '6', 'Minute': '0', 'Year': '2016', 'Month': '10', 'Day': '18'}, attributes={'PubStatus': 'pubmed'}), DictElement({'Hour': '6', 'Minute': '0', 'Year': '2016', 'Month': '10', 'Day': '18'}, attributes={'PubStatus': 'medline'})]
ArticleIdList
	 [StringElement('27749938', attributes={'IdType': 'pubmed'}), StringElement('10.1371/journal.pone.0164685', attributes={'IdType': 'doi'}), StringElement('PONE-D-16-23217', attributes={'IdType': 'pii'})]
PublicationStatus
	 epublish


The key `'MedlineCitation'` maps into another dictionary. In that dictionary, most of the information is contained under the key `'Article'`. To minimize the clutter, below we show the contents of `'MedlineCitation'` excluding its `'Article'` member, and below that we then show the contents of `'Article'`.

In [21]:
print_dict( { k:v for k,v in q[0][ 'MedlineCitation' ].items() if k!='Article' } )

OtherID
	 []
DateRevised
	 {'Year': '2016', 'Month': '10', 'Day': '18'}
GeneralNote
	 []
OtherAbstract
	 []
DateCreated
	 {'Year': '2016', 'Month': '10', 'Day': '17'}
SpaceFlightMission
	 []
MedlineJournalInfo
	 {'Country': 'United States', 'MedlineTA': 'PLoS One', 'NlmUniqueID': '101285081', 'ISSNLinking': '1932-6203'}
CitationSubset
	 []
PMID
	 27749938
KeywordList
	 []


In [22]:
print_dict( q[0][ 'MedlineCitation' ][ 'Article' ] )

ArticleTitle
	 Dynamical Mapping of Anopheles darlingi Densities in a Residual Malaria Transmission Area of French Guiana by Using Remote Sensing and Meteorological Data.
Abstract
	 {'AbstractText': [StringElement("Local variation in the density of Anopheles mosquitoes and the risk of exposure to bites are essential to explain the spatial and temporal heterogeneities in the transmission of malaria. Vector distribution is driven by environmental factors. Based on variables derived from satellite imagery and meteorological observations, this study aimed to dynamically model and map the densities of Anopheles darlingi in the municipality of Saint-Georges de l'Oyapock (French Guiana). Longitudinal sampling sessions of An. darlingi densities were conducted between September 2012 and October 2014. Landscape and meteorological data were collected and processed to extract a panel of variables that were potentially related to An. darlingi ecology. Based on these data, a robust methodology was f

A paper's abstract can therefore be accessed with:

In [23]:
{ int(q[0]['MedlineCitation']['PMID']) : str(q[0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0]) }

{27749938: "Local variation in the density of Anopheles mosquitoes and the risk of exposure to bites are essential to explain the spatial and temporal heterogeneities in the transmission of malaria. Vector distribution is driven by environmental factors. Based on variables derived from satellite imagery and meteorological observations, this study aimed to dynamically model and map the densities of Anopheles darlingi in the municipality of Saint-Georges de l'Oyapock (French Guiana). Longitudinal sampling sessions of An. darlingi densities were conducted between September 2012 and October 2014. Landscape and meteorological data were collected and processed to extract a panel of variables that were potentially related to An. darlingi ecology. Based on these data, a robust methodology was formed to estimate a statistical predictive model of the spatial-temporal variations in the densities of An. darlingi in Saint-Georges de l'Oyapock. The final cross-validated model integrated two landscap

A paper for which no abstract is available will simply not contain the `'Abstract'` key in its `'Article'` dictionary:

In [24]:
print_dict( Entrez.read( Entrez.efetch(db="pubmed", id='17782550', retmode="xml") )[0]['MedlineCitation']['Article'] )

ArticleTitle
	 EVOLUTION OF LOCOMOTIVES IN AMERICA.
PublicationTypeList
	 [StringElement('Journal Article', attributes={'UI': 'D016428'})]
Journal
	 {'ISOAbbreviation': 'Science', 'Title': 'Science (New York, N.Y.)', 'JournalIssue': DictElement({'Issue': '3', 'PubDate': {'Year': '1880', 'Month': 'Jul', 'Day': '17'}, 'Volume': '1'}, attributes={'CitedMedium': 'Print'}), 'ISSN': StringElement('0036-8075', attributes={'IssnType': 'Print'})}
ArticleDate
	 []
ELocationID
	 []
Pagination
	 {'MedlinePgn': '35'}
Language
	 ['ENG']


Some of the ids in our dataset refer to books from the [NCBI Bookshelf](http://www.ncbi.nlm.nih.gov/books/), a collection of freely available, downloadable, on-line versions of selected biomedical books. For such ids, `Entrez.efetch()` returns a slightly different structure, where the keys `[u'BookDocument', u'PubmedBookData']` take the place of the `[u'MedlineCitation', u'PubmedData']` keys we saw above.

Here is an example of the data we obtain for the id corresponding to the book [The Social Biology of Microbial Communities](http://www.ncbi.nlm.nih.gov/books/NBK114831/):

In [25]:
r = Entrez.read( Entrez.efetch(db="pubmed", id='24027805', retmode="xml") )

In [26]:
print_dict( r[0][ 'PubmedBookData' ] )

History
	 [DictElement({'Hour': '6', 'Minute': '0', 'Year': '2013', 'Month': '9', 'Day': '13'}, attributes={'PubStatus': 'pubmed'}), DictElement({'Hour': '6', 'Minute': '0', 'Year': '2013', 'Month': '9', 'Day': '13'}, attributes={'PubStatus': 'medline'}), DictElement({'Hour': '6', 'Minute': '0', 'Year': '2013', 'Month': '9', 'Day': '13'}, attributes={'PubStatus': 'entrez'})]
ArticleIdList
	 [StringElement('24027805', attributes={'IdType': 'pubmed'})]
PublicationStatus
	 ppublish


In [27]:
print_dict( r[0][ 'BookDocument' ] )

Sections
	 [{'Section': [], 'SectionTitle': StringElement('THE NATIONAL ACADEMIES', attributes={'book': 'nap13500', 'part': 'fm.s1'})}, {'Section': [], 'SectionTitle': StringElement('PLANNING COMMITTEE FOR A WORKSHOP ON THE MICROBIOME IN HEALTH AND DISEASE', attributes={'book': 'nap13500', 'part': 'fm.s2'})}, {'Section': [], 'SectionTitle': StringElement('FORUM ON MICROBIAL THREATS', attributes={'book': 'nap13500', 'part': 'fm.s3'})}, {'Section': [], 'SectionTitle': StringElement('BOARD ON GLOBAL HEALTH', attributes={'book': 'nap13500', 'part': 'fm.s5'})}, {'Section': [], 'SectionTitle': StringElement('Reviewers', attributes={'book': 'nap13500', 'part': 'fm.s7'})}, {'Section': [], 'SectionTitle': StringElement('Acknowledgments', attributes={'book': 'nap13500', 'part': 'fm.ack'})}, {'Section': [], 'SectionTitle': StringElement('Workshop Overview', attributes={'book': 'nap13500', 'part': 'workshop'})}, {'Section': [], 'SectionTitle': StringElement('Appendixes', attributes={'book': 'nap13

In a book from the NCBI Bookshelf, its abstract can then be accessed as such:

In [28]:
{ int(r[0]['BookDocument']['PMID']) : str(r[0]['BookDocument']['Abstract']['AbstractText'][0]) }

{24027805: 'On March 6 and 7, 2012, the Institute of Medicine’s (IOM’s) Forum on Microbial Threats hosted a public workshop to explore the emerging science of the “social biology” of microbial communities. Workshop presentations and discussions embraced a wide spectrum of topics, experimental systems, and theoretical perspectives representative of the current, multifaceted exploration of the microbial frontier. Participants discussed ecological, evolutionary, and genetic factors contributing to the assembly, function, and stability of microbial communities; how microbial communities adapt and respond to environmental stimuli; theoretical and experimental approaches to advance this nascent field; and potential applications of knowledge gained from the study of microbial communities for the improvement of human, animal, plant, and ecosystem health and toward a deeper understanding of microbial diversity and evolution.'}

### Abstracts dataset

We can now assemble a dataset mapping paper ids to their abstracts.

In [29]:
Abstracts_file = 'data/' + search_term + '__Abstracts.pkl.bz2'

In [30]:
import http.client
from collections import deque

if os.path.exists( Abstracts_file ):
    Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )
else:
    # `Abstracts` will be incrementally assembled, by performing multiple queries,
    # each returning at most `retrieve_per_query` entries.
    Abstracts = deque()
    retrieve_per_query = 500
    
    print('Fetching Abstracts of results: ')
    for start in range( 0, len(Ids), retrieve_per_query ):
        if (start % 10000 == 0):
            print('')
            print(start, end='')
        else:
            print('.', end='')
        
        # build comma separated string with the ids at indexes [start, start+retrieve_per_query)
        query_ids = ','.join( [ str(id) for id in Ids[ start : start+retrieve_per_query ] ] )
        
        # issue requests to the server, until we get the full amount of data we expect
        while True:
            try:
                s = Entrez.read( Entrez.efetch(db="pubmed", id=query_ids, retmode="xml" ) )
            except http.client.IncompleteRead:
                print('r', end='')
                continue
            break
        
        i = 0
        for p in s:
            abstr = ''
            if 'MedlineCitation' in p:
                pmid = p['MedlineCitation']['PMID']
                if 'Abstract' in p['MedlineCitation']['Article']:
                    abstr = p['MedlineCitation']['Article']['Abstract']['AbstractText'][0]
            elif 'BookDocument' in p:
                pmid = p['BookDocument']['PMID']
                if 'Abstract' in p['BookDocument']:
                    abstr = p['BookDocument']['Abstract']['AbstractText'][0]
            else:
                raise Exception('Unrecognized record type, for id %d (keys: %s)' % (Ids[start+i], str(p.keys())) )
            
            Abstracts.append( (int(pmid), str(abstr)) )
            i += 1
    
    # Save Abstracts, as a dictionary indexed by Ids
    Abstracts = dict( Abstracts )
    
    pickle.dump( Abstracts, bz2.BZ2File( Abstracts_file, 'wb' ) )

Fetching Abstracts of results: 

0...................
10000...................
20000...................
30000...................
40000...................
50000...................
60000..............

Taking a look at one paper's abstract:

In [31]:
Abstracts[27749938]

"Local variation in the density of Anopheles mosquitoes and the risk of exposure to bites are essential to explain the spatial and temporal heterogeneities in the transmission of malaria. Vector distribution is driven by environmental factors. Based on variables derived from satellite imagery and meteorological observations, this study aimed to dynamically model and map the densities of Anopheles darlingi in the municipality of Saint-Georges de l'Oyapock (French Guiana). Longitudinal sampling sessions of An. darlingi densities were conducted between September 2012 and October 2014. Landscape and meteorological data were collected and processed to extract a panel of variables that were potentially related to An. darlingi ecology. Based on these data, a robust methodology was formed to estimate a statistical predictive model of the spatial-temporal variations in the densities of An. darlingi in Saint-Georges de l'Oyapock. The final cross-validated model integrated two landscape variables

## ELink: Searching for related items in NCBI Entrez

To understand how to obtain paper citations with Entrez, we will first assemble a small set of PubMed IDs, and then query for their citations.
To that end, we search here for papers published in the [PLOS Computational Biology](http://www.ploscompbiol.org/) journal (as before, having also the word "malaria" in either the title or abstract):

In [32]:
CA_search_term = search_term+'[TIAB] AND PLoS computational biology[JOUR]'
CA_ids = Entrez.read( Entrez.esearch( db="pubmed", term=CA_search_term ) )['IdList']
CA_ids

['27509368', '27043913', '26890485', '26764905', '25590612', '25187979', '24465196', '24465193', '24348235', '24244127', '24204241', '24146604', '24130474', '23874190', '23785271', '23637586', '23637585', '23093922', '22615546', '22511852']

In [33]:
CA_summ = {
    p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( CA_ids )) )
    }
CA_summ

{'22511852': ('Evolution of the multi-domain structures of virulence genes in the human malaria parasite, Plasmodium falciparum.',
  ['Buckee CO', 'Recker M'],
  '2012',
  'PLoS computational biology',
  '10.1371/journal.pcbi.1002451'),
 '22615546': ('A spatial model of mosquito host-seeking behavior.',
  ['Cummins B', 'Cortez R', 'Foppa IM', 'Walbeck J', 'Hyman JM'],
  '2012',
  'PLoS computational biology',
  '10.1371/journal.pcbi.1002500'),
 '23093922': ('The dynamics of naturally acquired immunity to Plasmodium falciparum infection.',
  ['Pinkevych M', 'Petravic J', 'Chelimo K', 'Kazura JW', 'Moormann AM', 'Davenport MP'],
  '2012',
  'PLoS computational biology',
  '10.1371/journal.pcbi.1002729'),
 '23637585': ('Biomarker discovery by sparse canonical correlation analysis of complex clinical phenotypes of tuberculosis and malaria.',
  ['Rousu J', 'Agranoff DD', 'Sodeinde O', 'Shawe-Taylor J', 'Fernandez-Reyes D'],
  '2013',
  'PLoS computational biology',
  '10.1371/journal.pcbi.1

Because we restricted our search to papers in an open-access journal, you can then follow their DOIs to freely access their PDFs at the journal's website.

We will now issue calls to `Entrez.elink()` using these PubMed IDs, to retrieve the IDs of papers that cite them.
The database from which the IDs will be retrieved is [PubMed Central](http://www.ncbi.nlm.nih.gov/pmc/), a free digital database of full-text scientific literature in the biomedical and life sciences.

A complete list of the kinds of links you can retrieve with `Entrez.elink()` can be found [here](http://eutils.ncbi.nlm.nih.gov/entrez/query/static/entrezlinks.html).

In [36]:
CA_citing = {
    id : Entrez.read( Entrez.elink(
            cmd = "neighbor",               # ELink command mode: "neighbor", returns
                                            #     a set of UIDs in `db` linked to the input UIDs in `dbfrom`.
            dbfrom = "pubmed",              # Database containing the input UIDs: PubMed
            db = "pmc",                     # Database from which to retrieve UIDs: PubMed Central
            LinkName = "pubmed_pmc_refs",   # Name of the Entrez link to retrieve: "pubmed_pmc_refs", gets
                                            #     "Full-text articles in the PubMed Central Database that cite the current articles"
            from_uid = id                   # input UIDs
            ) )
    for id in CA_ids
    }

CA_citing['22511852']

[{'DbFrom': 'pubmed', 'IdList': ['22511852'], 'LinkSetDb': [{'DbTo': 'pmc', 'LinkName': 'pubmed_pmc_refs', 'Link': [{'Id': '4862716'}, {'Id': '4825093'}, {'Id': '4726288'}, {'Id': '4707858'}, {'Id': '4677286'}, {'Id': '4384290'}, {'Id': '4364229'}, {'Id': '4270465'}, {'Id': '3986854'}, {'Id': '3794903'}, {'Id': '3778436'}, {'Id': '3726600'}]}], 'LinkSetDbHistory': [], 'ERROR': []}]

We have in `CA_citing[paper_id][0]['LinkSetDb'][0]['Link']` the list of papers citing `paper_id`. To get it as just a list of ids, we can do

In [40]:
cits = [ l['Id'] for l in CA_citing['22511852'][0]['LinkSetDb'][0]['Link'] ]
cits

['4862716',
 '4825093',
 '4726288',
 '4707858',
 '4677286',
 '4384290',
 '4364229',
 '4270465',
 '3986854',
 '3794903',
 '3778436',
 '3726600']

However, one more step is needed, as what we have now are PubMed Central IDs, and not PubMed IDs. Their conversion can be achieved through an additional call to `Entrez.elink()`:

In [41]:
cits_pm = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=",".join(cits)) )
cits_pm

[{'DbFrom': 'pmc', 'IdList': ['4862716', '4825093', '4726288', '4707858', '4677286', '4384290', '4364229', '4270465', '3986854', '3794903', '3778436', '3726600'], 'LinkSetDb': [{'DbTo': 'pubmed', 'LinkName': 'pmc_pubmed', 'Link': [{'Id': '26883585'}, {'Id': '26804201'}, {'Id': '26741401'}, {'Id': '26674193'}, {'Id': '26657042'}, {'Id': '25759421'}, {'Id': '25626688'}, {'Id': '25521112'}, {'Id': '24674301'}, {'Id': '24130474'}, {'Id': '24062941'}, {'Id': '23922996'}]}], 'LinkSetDbHistory': [], 'ERROR': []}]

In [42]:
ids_map = { pmc_id : link['Id'] for (pmc_id,link) in zip(cits_pm[0]['IdList'], cits_pm[0]['LinkSetDb'][0]['Link']) }
ids_map

{'3726600': '23922996',
 '3778436': '24062941',
 '3794903': '24130474',
 '3986854': '24674301',
 '4270465': '25521112',
 '4364229': '25626688',
 '4384290': '25759421',
 '4677286': '26657042',
 '4707858': '26674193',
 '4726288': '26741401',
 '4825093': '26804201',
 '4862716': '26883585'}

And to check these papers:

In [43]:
{   p['Id'] : ( p['Title'], p['AuthorList'], p['PubDate'][:4], p['FullJournalName'], p.get('DOI', '') )
    for p in Entrez.read( Entrez.esummary(db="pubmed", id=','.join( ids_map.values() )) )
    }

{'23922996': ('Plasmodium falciparum var gene expression homogeneity as a marker of the host-parasite relationship under different levels of naturally acquired immunity to malaria.',
  ['Warimwe GM', 'Recker M', 'Kiragu EW', 'Buckee CO', 'Wambua J', 'Musyoki JN', 'Marsh K', 'Bull PC'],
  '2013',
  'PloS one',
  '10.1371/journal.pone.0070467'),
 '24062941': ('The antigenic switching network of Plasmodium falciparum and its implications for the immuno-epidemiology of malaria.',
  ['Noble R', 'Christodoulou Z', 'Kyes S', 'Pinches R', 'Newbold CI', 'Recker M'],
  '2013',
  'eLife',
  '10.7554/eLife.01074'),
 '24130474': ('A network approach to analyzing highly recombinant malaria parasite genes.',
  ['Larremore DB', 'Clauset A', 'Buckee CO'],
  '2013',
  'PLoS computational biology',
  '10.1371/journal.pcbi.1003268'),
 '24674301': ('Plasmodium falciparum antigenic variation: relationships between widespread endothelial activation, parasite PfEMP1 expression and severe malaria.',
  ['Abdi A

### Citations dataset

We have now seen all the steps required to assemble a dataset of citations to each of the papers in our dataset.

In [24]:
Citations_file = 'data/' + search_term + '__Citations.pkl.bz2'
Citations = []

At least one server query will be issued per paper in `Ids`. Because NCBI allows for at most 3 queries per second (see [here](http://biopython.org/DIST/docs/api/Bio.Entrez-pysrc.html#_open)), this dataset will take a long time to assemble. Should you need to interrupt it for some reason, or the connection fail at some point, it is safe to just rerun the cell below until all data is collected.

In [27]:
import http.client

if Citations == [] and os.path.exists( Citations_file ):
    Citations = pickle.load( bz2.BZ2File( Citations_file, 'rb' ) )

if len(Citations) < len(Ids):
    
    i = len(Citations)
    checkpoint = len(Ids) / 10 + 1      # save to hard drive at every 10% of Ids fetched
    
    for pm_id in Ids[i:]:               # either starts from index 0, or resumes from where we previously left off
        
        while True:
            try:
                # query for papers archived in PubMed Central that cite the paper with PubMed ID `pm_id`
                c = Entrez.read( Entrez.elink( dbfrom = "pubmed", db="pmc", LinkName = "pubmed_pmc_refs", id=str(pm_id) ) )
                
                c = c[0]['LinkSetDb']
                if len(c) == 0:
                    # no citations found for the current paper
                    c = []
                else:
                    c = [ l['Id'] for l in c[0]['Link'] ]
                    
                    # convert citations from PubMed Central IDs to PubMed IDs
                    p = []
                    retrieve_per_query = 500
                    for start in range( 0, len(c), retrieve_per_query ):
                        query_ids = ','.join( c[start : start+retrieve_per_query] )
                        r = Entrez.read( Entrez.elink( dbfrom="pmc", db="pubmed", LinkName="pmc_pubmed", from_uid=query_ids ) )
                        # select the IDs. If no matching PubMed ID was found, [] is returned instead
                        p.extend( [] if r[0]['LinkSetDb']==[] else [ int(link['Id']) for link in r[0]['LinkSetDb'][0]['Link'] ] )
                    c = p
            
            except http.client.BadStatusLine:
                # Presumably, the server closed the connection before sending a valid response. Retry until we have the data.
                print('r')
                continue
            break
        
        Citations.append( (pm_id, c) )
        if (i % 10000 == 0):
            print('')
            print(i, end='')
        if (i % 100 == 0):
            print('.', end='')
        i += 1
        
        if i % checkpoint == 0:
            print('\tsaving at checkpoint', i)
            pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )
    
    print('\n done.')
    
    # Save Citations, as a dictionary indexed by Ids
    Citations = dict( Citations )
    
    pickle.dump( Citations, bz2.BZ2File( Citations_file, 'wb' ) )

......................................................................................
30000....................................................................................................
40000....................................................................................................
50000....................................................................................................
60000.......................................................................
 done.


To see that we have indeed obtained the data we expected, you can match the ids below, with the ids listed at the end of last section.

In [29]:
Citations[24130474]

[27306566, 26456841, 25521112, 25368109, 25303095, 25122340]

## Where do we go from here?

Running the code above generates multiple local files, containing the datasets we'll be working with. Loading them into memory is a matter of just issuing a call like<br>
``data = pickle.load( bz2.BZ2File( data_file, 'rb' ) )``.

The Entrez module will therefore no longer be needed, unless you wish to extend your data processing with additional information retrieved from PubMed.

Should you be interested in looking at alternative ways to handle the data, have a look at the [sqlite3](http://docs.python.org/3/library/sqlite3.html) module included in Python's standard library, or [Pandas](http://pandas.pydata.org/), the Python Data Analysis Library.